In [1]:
import os
from pathlib import Path
import sys
sys.path.append('src')

from src.seg_openseed import OpenSeeDSeg
from src.dataset import collate_fn
from src.dataset.nutrition5k_dataset import Nutrition5kDataset, make_dataset
from src.dataset import Metadata
from OpenSeeD.utils.visualizer import Visualizer

from detectron2.data import MetadataCatalog
from detectron2.utils.colormap import random_color
from detectron2.structures import BitMasks

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from torchvision.transforms.functional import to_pil_image

metadatas_path = '/srv/datasets2/nutrition5k_dataset/metadata/dish_metadata_cafe1.csv'
imgs_dir = '/srv/datasets2/nutrition5k_dataset/imagery/realsense_overhead'
splits_path = '/srv/datasets2/nutrition5k_dataset/dish_ids/splits/depth_train_ids.txt'
splits_test_path = '/srv/datasets2/nutrition5k_dataset/dish_ids/splits/depth_test_ids.txt'
splits =  [line.rstrip() for line in open(splits_path,'r').readlines()]

/home/parinayok/.pyenv/versions/miniconda3-latest/envs/nutrition5k/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = make_dataset(None,imgs_dir,metadatas_path,splits_path,splits_test_path)

In [3]:
dataloader = {x: DataLoader(dataset[x], batch_size=8, num_workers=8, shuffle=False, collate_fn=collate_fn) for x in ['train','test']}

In [13]:
from src.model import FPNCrossSwinMultiMask, Regressor
model = FPNCrossSwinMultiMask(768, 12, 2, 12, 4.,regressor=Regressor(3120,3120))

Some weights of the model checkpoint at microsoft/swin-tiny-patch4-window7-224 were not used when initializing SwinModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing SwinModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SwinModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at microsoft/swin-tiny-patch4-window7-224 were not used when initializing SwinModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing SwinModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining m

In [8]:
from torchvision import transforms
for i, batch in enumerate(dataloader['train']):
  img=batch['rgb_img']
  depth=batch['depth_img']
  mask=batch['mask']
  mask_all=batch['mask_all']
  break
resize = transforms.Resize((256, 352))
img = resize(img)
depth = resize(depth)
mask_all = resize(mask_all)
mask = mask.unsqueeze(1)
mask = resize(mask)
mask = mask.squeeze(1)

In [14]:
output = model(img, depth, mask=mask_all)